In [1]:
import pandas as pd
import numpy as np

In [2]:
cbpd = pd.read_csv("../derivatives/CBPD_data_DMD_2023.11.14.csv")
nwbd = pd.read_csv("../derivatives/NWBD_BrannonBehaviorLogs_11.5.2019_MN.csv")

In [3]:
cbpd['parent_edu'] = [np.nanmean([cbpd['parent1_edu'][i], cbpd['parent2_edu'][i]]) for i in np.arange(cbpd.shape[0])] 

<ipython-input-3-87c3df96cd06>:1: RuntimeWarning: Mean of empty slice
  cbpd['parent_edu'] = [np.nanmean([cbpd['parent1_edu'][i], cbpd['parent2_edu'][i]]) for i in np.arange(cbpd.shape[0])]


In [4]:
timepoint1 = [i for i in np.arange(cbpd.shape[0]) if '_' not in cbpd.record_id.values[i]]
timepoint2 = [i for i in np.arange(cbpd.shape[0]) if '_2' in cbpd.record_id.values[i]]
timepoint3 = [i for i in np.arange(cbpd.shape[0]) if '_3' in cbpd.record_id.values[i]]

In [5]:
cbpdvariables = ['record_id', 'age_scan', 'male',  't1_rating_avg', 'parent_edu',  'income_median']
nwbdvariables = ['record_id', 'age_scan', 'male', 't1_rating_avg']

In [6]:
cbpd = cbpd.loc[:, cbpdvariables]
nwbd = nwbd.loc[:, nwbdvariables]
nwbd['parent_edu'] = np.repeat(None, nwbd.shape[0])
nwbd['income_median'] = np.repeat(None, nwbd.shape[0])

In [23]:
for i, timepoint in enumerate([timepoint1, timepoint2, timepoint3]):
    cbpd_t = cbpd.iloc[timepoint,:]
    if i == 0:
        cbpd_t = pd.concat([cbpd_t, nwbd]).reset_index(drop=True)
    cbpd_t['fsid'] = ['sub-'+x.split('_')[0] for x in cbpd_t.record_id]
    cbpd_t = cbpd_t.dropna(subset=['age_scan', 'male', 't1_rating_avg'])
    cbpd_t['age_scan'] = [float(x) for x in cbpd_t.age_scan]
    cbpd_t['age_square_scan'] = [x**2 for x in cbpd_t.age_scan]
    for var in ['age', 'parent_edu', 'income_median']:
        if var=='age':
            df_var = cbpd_t.loc[:, ['fsid', 'age_scan', 'age_square_scan', 'male', 't1_rating_avg']]#.dropna('age_scan')#how='any')
            df_var.to_csv(f'../derivatives/t{str(i+1)}_nonormalize.table.dat', index=False)
            for v in ['age_scan', 'age_square_scan', 't1_rating_avg']:
                df_var[v] = (df_var[v] - df_var[v].mean())/df_var[v].std()
            df_var.to_csv(f'../derivatives/t{str(i+1)}_normalize.table.dat', index=False)
        else:
            df_var = cbpd_t.loc[:, ['fsid', var, 'age_scan', 'male', 't1_rating_avg']].dropna(how='any')
            df_var.to_csv(f'../derivatives/t{str(i+1)}_{var}_nonormalize.table.dat', index=False)
            for v in ['age_scan', 't1_rating_avg', var]:
                df_var[v] = (df_var[v] - df_var[v].mean())/df_var[v].std()
            df_var.to_csv(f'../derivatives/t{str(i+1)}_{var}_normalize.table.dat', index=False)

<ipython-input-23-cab68aa025d8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbpd_t['fsid'] = ['sub-'+x.split('_')[0] for x in cbpd_t.record_id]
<ipython-input-23-cab68aa025d8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cbpd_t['fsid'] = ['sub-'+x.split('_')[0] for x in cbpd_t.record_id]
